In [1]:
import torch #pytorch
import torch.nn as nn #neural network
import torch.optim as optim #optimizer
import torch.nn.functional as F #Activation function
from torch.utils.data import DataLoader #Batch spliter
from torchvision.datasets import MNIST #datasets for computer vision
import torchvision.transforms as transforms #Data transformation

In [14]:
#Create Fully Connected layer
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.fc1=nn.Linear(input_size,50)
        self.fc2=nn.Linear(50,100)
        self.fc3=nn.Linear(100,num_classes)

    def forward(self,X):
        fc1_output=F.relu(self.fc1(X))
        fc2_output=F.relu(self.fc2(fc1_output))
        final_output=self.fc3(fc2_output)
        return final_output


In [15]:
#Hyperparameters
input_size=784
num_classes=10
epochs=1
learning_rate=0.001
batch_size=128

In [16]:
#Data processing
trainData=MNIST(root="dataset/",train=True,transform=transforms.ToTensor(),download=True)
train_batches=DataLoader(dataset=trainData,batch_size=batch_size,shuffle=True)
testData=MNIST(root="dataset/",train=False,transform=transforms.ToTensor(),download=True)
test_batches=DataLoader(dataset=testData,batch_size=batch_size,shuffle=True)

In [17]:
import torch.cuda
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
model=NN(input_size=input_size,num_classes=num_classes)

In [19]:
creterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [23]:

for i in range(epochs):
  for batch_idx,(data,targets) in enumerate(train_batches):
    data=data.to(device)
    targets=targets.to(targets)

    data=data.reshape(data.shape[0],-1)

    predicted=model(data)
    loss=creterion(predicted,targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()


NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

In [25]:
def check_accuracy(batches,model):
  num_samples=0
  num_correct=0
  model.eval()

  with torch.no_grad():
    for data,targets in batches:
      data=data.to(device)
      targets=targets.to(device)
      data=data.reshape(data.shape[0],-1)

      output=model(data)
      _,predict=output.max(1)

      num_correct+=(targets==predict).sum()
      num_samples+=len(targets)

  model.train()
  return num_correct/num_samples


In [27]:
print(check_accuracy(test_batches,model))

tensor(0.9460)


In [38]:
def predict(batches,model):
  model.eval()
  with torch.no_grad():
    total_output=[]
    for data,targets in batches:
      data=data.to(device)
      targets=targets.to(device)
      data=data.reshape(data.shape[0],-1)

      output=model(data)
      _,predict=output.max(1)
      total_output.append(list(predict))
    model.train()
    return total_output

In [39]:
print(predict(test_batches,model))

[[tensor(3), tensor(9), tensor(1), tensor(6), tensor(9), tensor(5), tensor(7), tensor(9), tensor(7), tensor(2), tensor(6), tensor(6), tensor(8), tensor(7), tensor(0), tensor(3), tensor(7), tensor(4), tensor(7), tensor(9), tensor(1), tensor(7), tensor(3), tensor(1), tensor(8), tensor(1), tensor(7), tensor(2), tensor(6), tensor(5), tensor(3), tensor(6), tensor(8), tensor(4), tensor(7), tensor(3), tensor(8), tensor(9), tensor(3), tensor(3), tensor(6), tensor(4), tensor(4), tensor(1), tensor(1), tensor(5), tensor(1), tensor(9), tensor(6), tensor(3), tensor(7), tensor(0), tensor(1), tensor(4), tensor(3), tensor(0), tensor(2), tensor(3), tensor(3), tensor(0), tensor(9), tensor(4), tensor(9), tensor(0), tensor(6), tensor(7), tensor(3), tensor(3), tensor(7), tensor(6), tensor(5), tensor(9), tensor(2), tensor(4), tensor(7), tensor(1), tensor(5), tensor(9), tensor(7), tensor(8), tensor(4), tensor(5), tensor(6), tensor(4), tensor(9), tensor(6), tensor(9), tensor(3), tensor(3), tensor(2), tensor(0

In [41]:
def printTargets(test_batches):
  total_targets=[]
  for data,targets in test_batches:
    total_targets.append(targets)
  print(total_targets)
printTargets(test_batches)

[tensor([8, 9, 3, 8, 1, 4, 0, 4, 3, 8, 1, 2, 1, 2, 1, 7, 4, 1, 9, 1, 5, 0, 0, 6,
        9, 7, 3, 3, 9, 5, 1, 4, 3, 6, 3, 8, 2, 6, 6, 5, 1, 4, 6, 0, 2, 6, 8, 2,
        4, 9, 0, 1, 8, 3, 9, 0, 6, 5, 4, 5, 6, 8, 3, 3, 1, 6, 9, 3, 5, 9, 2, 7,
        8, 0, 8, 9, 3, 3, 1, 7, 5, 1, 2, 0, 6, 8, 6, 9, 2, 3, 3, 1, 2, 5, 3, 1,
        6, 9, 7, 3, 4, 2, 9, 7, 7, 5, 7, 6, 4, 0, 9, 6, 5, 5, 1, 7, 0, 3, 7, 0,
        0, 4, 6, 0, 9, 1, 0, 8]), tensor([3, 0, 4, 1, 2, 8, 6, 7, 5, 0, 1, 4, 3, 6, 1, 8, 5, 0, 0, 3, 5, 4, 1, 1,
        0, 0, 6, 9, 5, 3, 5, 1, 3, 9, 9, 5, 4, 8, 6, 9, 4, 0, 5, 1, 0, 6, 2, 1,
        4, 8, 0, 6, 4, 8, 4, 3, 4, 9, 1, 2, 0, 0, 3, 8, 5, 7, 7, 6, 3, 5, 8, 1,
        6, 4, 5, 9, 2, 8, 2, 4, 1, 7, 5, 0, 3, 0, 1, 4, 4, 0, 5, 9, 4, 1, 4, 8,
        9, 3, 0, 0, 4, 7, 8, 9, 6, 4, 6, 9, 0, 8, 2, 3, 0, 0, 2, 6, 8, 2, 9, 3,
        1, 6, 2, 5, 0, 4, 4, 7]), tensor([5, 8, 9, 6, 4, 4, 0, 6, 6, 1, 2, 7, 4, 9, 2, 1, 7, 3, 0, 2, 6, 7, 3, 7,
        6, 6, 0, 5, 6, 1, 1, 9, 0, 7, 4, 1, 8, 0, 8